<a href="https://colab.research.google.com/github/spatank/CIS-530/blob/master/Homework%206/hw6_skeleton_RIF_edit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download and unzip files

In [7]:
#Download and unzip files
!pip3 install scikit-learn
!wget http://computational-linguistics-class.org/homework/nn-lms/cities_test.txt
!wget http://computational-linguistics-class.org/homework/nn-lms/cities_val.zip
!wget http://computational-linguistics-class.org/homework/nn-lms/cities_train.zip
!sudo apt-get install unzip
!unzip cities_val.zip 
!unzip cities_train.zip 
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.0.1-cp36-cp36m-linux_x86_64.whl
!pip3 install torch torchvision
  
import torch
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

--2020-02-22 21:15:51--  http://computational-linguistics-class.org/homework/nn-lms/cities_test.txt
Resolving computational-linguistics-class.org (computational-linguistics-class.org)... 185.199.110.153
Connecting to computational-linguistics-class.org (computational-linguistics-class.org)|185.199.110.153|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10973 (11K) [text/plain]
Saving to: ‘cities_test.txt’

cities_test.txt     100%[===================>]  10.72K  --.-KB/s    in 0s      

2020-02-22 21:15:51 (394 MB/s) - ‘cities_test.txt’ saved [10973/10973]

--2020-02-22 21:15:53--  http://computational-linguistics-class.org/homework/nn-lms/cities_val.zip
Resolving computational-linguistics-class.org (computational-linguistics-class.org)... 185.199.110.153
Connecting to computational-linguistics-class.org (computational-linguistics-class.org)|185.199.110.153|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7558 (7.4K) [application/zip]
Sav

In [2]:
print(print(torch.__version__))

1.0.1
None


# Verify file download

In [9]:
#Verfiy file download
!head train/af.txt
!printf "\n"
!head val/af.txt
!printf "\n"
!head cities_test.txt
!printf "\n"
#Verify CUDA acceleration should print cuda:0
print(device)

garavshakh
kalishli
sarban kelay
dahan-e surunah
jahannum
morafeno
iskatel
zeri-chinar
taftebole
dzhafarabat

sikhtopa
laferronay
cham ghafur
souk tafetecht
panqash
siah sar bala
barah khan kili
vetob
kacari
kolga'i-i-bala

imam bakhsh amar
firoz laghari
glufishevo
qal`eh-ye ghazanfarkhani
nikolinalevada
kolkko
mbrostar i fierit
kharabanan-e sofla
stein bei nurnberg
jahangir bara

cuda:0


# Paths

In [0]:
root = '/content/gdrive/Shared drives/cis530_hw6/'
vectors_root = '/content/gdrive/Shared drives/vectors/'

# Mount google drive

In [27]:
# Mount your google drive. 
# Use this to save your PyTorch model for submission
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/Shared drives/cis530_hw6/'
!ls
#!mkdir /content/gdrive/My\ Drive/cis530_hw6
#Test drive access. 
#You should have a test.txt under a new folder cis530_hw6 in your Google drive
with open('/content/gdrive/Shared drives/cis530_hw6/test.txt', 'w') as f:
  f.write('This is a test!')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/Shared drives/cis530_hw6
data		    hw6_skeleton_RIF_edit.ipynb  skeleton  text_file.txt  val
hw6_skeleton.ipynb  hw6_SP_edit.ipynb		 test.txt  train


# Baby names tutorial

In [29]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

def findFiles(path): return glob.glob(path)

print("this is findFiles: ", findFiles(root + 'data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8', errors='ignore').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles(root + 'data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

this is findFiles:  ['/content/gdrive/Shared drives/cis530_hw6/data/names/Arabic.txt', '/content/gdrive/Shared drives/cis530_hw6/data/names/Chinese.txt', '/content/gdrive/Shared drives/cis530_hw6/data/names/Irish.txt', '/content/gdrive/Shared drives/cis530_hw6/data/names/Czech.txt', '/content/gdrive/Shared drives/cis530_hw6/data/names/German.txt', '/content/gdrive/Shared drives/cis530_hw6/data/names/Dutch.txt', '/content/gdrive/Shared drives/cis530_hw6/data/names/English.txt', '/content/gdrive/Shared drives/cis530_hw6/data/names/Italian.txt', '/content/gdrive/Shared drives/cis530_hw6/data/names/French.txt', '/content/gdrive/Shared drives/cis530_hw6/data/names/Greek.txt', '/content/gdrive/Shared drives/cis530_hw6/data/names/Japanese.txt', '/content/gdrive/Shared drives/cis530_hw6/data/names/Scottish.txt', '/content/gdrive/Shared drives/cis530_hw6/data/names/Vietnamese.txt', '/content/gdrive/Shared drives/cis530_hw6/data/names/Portuguese.txt', '/content/gdrive/Shared drives/cis530_hw6/da

## test category lines

In [30]:
print(category_lines['Italian'][:5])

['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


# Write output files (all methods)


In [0]:
from datetime import datetime
import csv

In [0]:
## ADDED
def write_file(filename, results):
  '''
    Input: List ['this is an', 'example']
    Output: This overwirtes file every time, does not append
  '''
  dateTimeObj = datetime.now()
  results.insert(0,'******* New File ' + str(dateTimeObj) + '***********')
  with open(filename,'w') as f:
      for item in results:
          f.write(str(item) + '\n')

  f.close()

In [0]:
## ADDED
def append_to_output_file(filename, results):
  '''
    Input: List ['this is an', 'example']
    Output: Keeps appending to same file (does NOT overwrite)
  '''
  dateTimeObj = datetime.now()
  results.insert(0,'******* New File ' + str(dateTimeObj) + '***********')
  
  ## write to output file
  with open(filename,'a') as f:
      results.insert(0, "*******  Beginning of new File  ********")
      print(type(results))
      for item in results:
          f.write(str(item) + '\n')

  f.close()

In [0]:
## ADDED
def write_outfile(filename, results):
  '''
    Input: String
    Output: Keeps appending to same file (does NOT overwrite)
  '''
  dateTimeObj = datetime.now()

  with open(filename,'a') as f:
      f.write('******* Beginning of file ' + str(dateTimeObj) +  '********\n')
      f.write(results + '\n')

  f.close()

In [0]:
## ADDED
def write_outfile_with_modelname(filename, results, modelname):
  '''
    Input: String, modelname to be printed with results
    Output: Keeps appending to same file (does NOT overwrite)
  '''
  dateTimeObj = datetime.now()

  with open(filename,'a') as f:
      f.write(modelname + '\n')
      f.write('******* Beginning of file ' + str(dateTimeObj) +  '********\n')
      f.write(results + '\n')

  f.close()

# main_classify.py

In [0]:
#main_classify.py
import codecs
import math
import random
import string
import time
import numpy as np
import torch
from sklearn.metrics import accuracy_score

'''
Don't change these constants for the classification task.
You may use different copies for the sentence generation model.
'''
languages = ["af", "cn", "de", "fi", "fr", "in", "ir", "pk", "za"]
all_letters = string.ascii_letters + " .,;'"

'''
Returns the words of the language specified by reading it from the data folder
Returns the validation data if train is false and the train data otherwise.
Return: A nx1 array containing the words of the specified language
'''
def getWords(baseDir, lang, train = True):
    pass

'''
Returns a label corresponding to the language
For example it returns an array of 0s for af
Return: A nx1 array as integers containing index of the specified language in the "languages" array
'''
def getLabels(lang, length):
    pass

'''
Returns all the laguages and labels after reading it from the file
Returns the validation data if train is false and the train data otherwise.
You may assume that the files exist in baseDir and have the same names.
Return: X, y where X is nx1 and y is nx1
'''
def readData(baseDir, train=True):
    pass

'''
Convert a line/word to a pytorch tensor of numbers
Refer the tutorial in the spec
Return: A tensor corresponding to the given line
'''
def line_to_tensor(line):
    pass

'''
Returns the category/class of the output from the neural network
Input: Output of the neural networks (class probabilities)
Return: A tuple with (language, language_index)
        language: "af", "cn", etc.
        language_index: 0, 1, etc.
'''
def category_from_output(output):
    pass

'''
Get a random input output pair to be used for training 
Refer the tutorial in the spec
'''
def random_training_pair(X, y):
    pass

'''
Input: trained model, a list of words, a list of class labels as integers
Output: a list of class labels as integers
'''
def predict(model, X, y):
    pass

'''
Input: trained model, a list of words, a list of class labels as integers
Output: The accuracy of the given model on the given input X and target y
'''
def calculateAccuracy(model, X, y):
    pass

'''
Train the model for one epoch/one training word.
Ensure that it runs within 3 seconds.
Input: X and y are lists of words as strings and classes as integers respectively
Returns: You may return anything
'''
def trainOneEpoch(model, criterion, optimizer, X, y):
    pass

'''
Use this to train and save your classification model. 
Save your model with the filename "model_classify"
'''
def run():
    pass


# models.py

In [0]:
#models.py
import torch.nn as nn
from torch.autograd import Variable
import torch
import torch.nn.functional as F

'''
Please add default values for all the parameters of __init__.
'''
class CharRNNClassify(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        pass

    def forward(self, input, hidden=None):
        pass

    def init_hidden(self):
        pass